In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
 
plt.style.use('seaborn')
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# 경로 설정 (노트북 파일 기점으로 상대경로 혹은 절대경로)
data = pd.read_csv('train.csv')
#data = pd.read_csv(r'C:\Users\Ki_Yoon_Yoo\Desktop\ToBigs/train.csv')

In [ ]:
# Display first five rows 
data.head(5)

In [ ]:
### Data Exploration
 
# 어떤 종류의 feature 가 있는지 살펴봅시다
print('Types of Columns: ')
print(data.columns.values)

In [ ]:
# 연속 변수 통계치 확인 
print(data.describe())

In [ ]:
# 각 피쳐의 data type 을 살펴봅시다 
print(data.dtypes)

In [ ]:
### Null 값 처리 : Method 1 
 
# Null 값 개수 복습!
data.isnull().sum()

In [ ]:
# Age, Cabin, Embarked 세 가지 feuture 에서 결측치가 존재하네요. 특히, 'Cabin' 의 경우 대부분의 데이터가 누락되었습니다. 'Cabin' 은 Cabin Number 로 좌석 위치를 의미하는 것 같습니다. A석, B석, C석 .. F 석 따라서 생존율에 영향이 있었을 것으로 보입니다. 다만, 결측치가 너무 많기 때문에 이 feature 를 활용하기 어려워보이네요. 
 
print(data['Cabin'].unique())
 
data.drop('Cabin',axis=1).head(3)

In [ ]:
### Null 값 처리 : Method 2 (통계치, 외부 정보 활용)  
 
# 좌석 위치에 대한 직접적인 정보를 이용하기 보다는 'Null 값 여부' 정보를 이용해봅시다. 
 
# Fill 'NA' into Null value 
data['Cabin'].fillna('NA', inplace=True)
print(data['Cabin'].unique())

In [ ]:
# Conditional statement to fill 'Null' or 'Non_Null'
data.loc[data['Cabin']=='NA','Cabin_Null'] = 'Null'
data.loc[data['Cabin']!='NA','Cabin_Null'] = 'Non-Null'
 
data['Cabin_Null'].head(3)
 
sns.countplot('Cabin_Null',hue='Survived',data=data)
 
data[['Cabin_Null','Survived']].groupby(['Cabin_Null']).count()
 
data[['Cabin_Null','Survived']].groupby(['Cabin_Null']).mean()

In [ ]:
#### Age
 
data['Age'].isnull().sum()
# 177/891 = 19.2 % 

In [ ]:
# 산술평균 이용
data['Age'].fillna(data['Age'].mean(), inplace=True)

In [ ]:
# 중간값 이용 
data['Age'].fillna(data['Age'].median(), inplace=True)

In [ ]:
# 탑승객의 이름은 다음과 같은 양식으로 기재되어있습니다. 
# Braund, Mr. Own Harris 의 Braund 는 first name / Owen Harris 는 last name 과 middle name 같군요
data['Name'].head()
 
for i in data:
    data['Initial']=data['Name'].str.extract('([A-Za-z]+)\.') # 온점(.) 뒤에 있는 이니셜 정보를 추출한뒤 저장합니다.
 
pd.crosstab(data['Initial'],data['Sex']).T.style.background_gradient(cmap='summer_r') #이니셜을 성별에 대해서 정렬

In [ ]:
data.loc[data['Initial'] == 'Master', 'Age'].head(10)
# 대부분 어려보이는데 29살은 뭘까요??

In [ ]:
# 나머지는 이니셜을 모두 Mr. / Miss / Mrs 로 처리합시다. 
# 여성의 이니셜의 경우, 이니셜의 의미에 가장 부합하는 것 같은 카테고리 (Miss 혹은 Mrs)로 임의로 나눱습니다. 
# 귀족 이니셜은 'Other' 로 따로 묶었습니다
data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Other','Other','Other'],inplace=True)

In [ ]:
# 이제 남은 것은 다음과 같은 이니셜 뿐입니다.
data['Initial'].unique()
 
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']= data.loc[data['Initial'] == 'Master', 'Age'].median()
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']= data.loc[data['Initial'] == 'Mr', 'Age'].median()
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']= data.loc[data['Initial'] == 'Mrs', 'Age'].median()
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']= data.loc[data['Initial'] == 'Miss', 'Age'].median()
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']= data.loc[data['Initial'] == 'Other', 'Age'].median()
 
print('Median Age for Master : ', data.loc[data['Initial'] == 'Master', 'Age'].median()) 
print('Median Age for Mr : ', data.loc[data['Initial'] == 'Mr', 'Age'].median()) 
print('Median Age for Mrs : ', data.loc[data['Initial'] == 'Mrs', 'Age'].median()) 
print('Median Age for Miss : ', data.loc[data['Initial'] == 'Miss', 'Age'].median()) 
print('Median Age for Other : ', data.loc[data['Initial'] == 'Other', 'Age'].median()) 

In [ ]:
### Categorical Feature 처리 
 
#### Nomial : 순서가 상관없는 Categorical
 
dummy_var = pd.get_dummies(data.Initial)
pd.get_dummies(data.Initial).head()

In [ ]:
pd.concat([data.drop(['Initial'], axis=1),dummy_var], axis=1).head()

In [ ]:
#### Ordinal : 순서가 상관 있는 Categorical
 
def transform_to_string(x):
    if x==1:
        return 'class1'
    elif x==2:
        return 'class2'
    else:
        return 'class3'

In [ ]:
data['Pclass'] = data['Pclass'].apply(transform_to_string)
data['Pclass'].head(10)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
mapper = [['class3',],['class2',],['class1',]]
ordinal_encoder.fit(mapper)

In [ ]:
ordinal_encoder.transform(np.array(data['Pclass']).reshape(-1,1))

In [ ]:
data['Pclass']

In [ ]:
## 과제 1:
#### Target Encoding: 각 피쳐 값의 target variable 평균으로 인코딩

In [ ]:
#### Mean 값 찾기
# Initial 변수를 기준으로 생존율(means)을 구해줍니다.
data['Initial_encoded'] = 0 
means = data.groupby('Initial')['Survived'].mean(); means

In [ ]:
#### DataFrame 에 저장
# map함수를 이용해 Initial 변수를 기준으로 Initial_encoded에 means를 넣어줍니다
data['Initial_encoded'] = data['Initial'].map(means)
data